# Sprint 7 · Webinar 22 · Data Analytics práctico (Proyecto: outliers, segmentación y GitHub)

**Duración:** 100 minutos  
**Modalidad:** Práctica guiada (proyecto paso a paso)  


## Fecha

Completa la información de la sesión:

- **Fecha:**  
- **Instructor/a:**  
- **Duración:** 100 minutos  


## Objetivos de la sesión práctica

Al finalizar esta sesión, la persona estudiante será capaz de:

1. **Generar y documentar** un dataset sintético realista (CSV) para un proyecto práctico.
2. Realizar **carga, limpieza y exploración** de datos (tipos, nulos, duplicados, calidad).
3. Construir **métricas por cliente** (agregaciones tipo RFM y KPIs operativos).
4. **Detectar y tratar valores atípicos** usando reglas estadísticas (IQR/Z-score) y criterio de negocio.
5. Crear **segmentos de clientes** con `if/elif/else` y con `apply()` usando funciones personalizadas.
6. Redactar un **Statistical Summary** (hallazgos clave + métricas + decisiones tomadas).
7. Empaquetar, versionar y **publicar el proyecto en GitHub** (estructura de repo + workflow).


## Metodología de la sesión

**Vamos a realizar nuestro primer proyecto para nuestro portafolio!**

Comparte con los estudoantes el dataset que vamos a utilizar. Más adelante encontrarás detalles sobre el dataset (Lo podés encontrar sobre la carpeta `datasets`).

- Cada estudiante replica el notebook en su propio espacio de Google Colab.
- La practica de Git incluirá crear un repositorio en GitHub y subir el notebook.
- Estudiar los comandos básicos de Git editando el archivo README.md.
- Estudiar los comandos básicos de Git subiendo el .csv del dataset.

## Agenda sugerida (100 minutos)

| Tiempo | Bloque | Contenido | Modalidad |
|---:|---|---|---|
| 10 min | Contexto del proyecto | Caso de uso, entregables y dataset | Guía |
| 15 min | Dataset (CSV) + carga | Generación, diccionario de datos, lectura | Live coding |
| 20 min | Calidad + EDA | Tipos, nulos, duplicados, distribuciones | Ejercicio guiado |
| 20 min | Métricas por cliente | Agregaciones y KPIs (RFM básico) | Ejercicio guiado |
| 20 min | Outliers | Detección (IQR/Z) + tratamiento por contexto | Ejercicio guiado |
| 10 min | Segmentación | `if` + `apply()` con función | Ejercicio guiado |
| 5 min | Statistical Summary | Plantilla + texto basado en KPIs | Guía |
| 10 min | Publicación en GitHub | Estructura, commits, push, Colab workflow | Guía práctica |


## Ejercicio 0 · Kickoff del proyecto (5 min)

En parejas (o individual), responde:

1. ¿Qué decisiones de negocio podríamos tomar con estos datos?
2. ¿Qué variables crees que tendrán outliers y por qué?
3. ¿Qué significa “segmento” para ti en este contexto (retención, VIP, riesgo)?

**Salida esperada:** 3–5 bullets con hipótesis iniciales.  
Luego validaremos (o corregiremos) esas hipótesis con datos.


## Proyecto práctico: “Clínica Agendada” (Customer Analytics)

Vas a trabajar como analista en una empresa que ofrece servicios de salud bajo agenda. El negocio quiere responder:

- ¿Cómo se comportan los clientes por plan, canal y ciudad?
- ¿Qué tan frecuentes son los **reembolsos** y cómo afectan los ingresos?
- ¿Existen **valores atípicos** (montos, tiempos de espera, edades) que distorsionen el análisis?
- ¿Podemos **segmentar** clientes para acciones (retención, cross-sell, incentivos)?

### Entregables del proyecto (lo que publicaremos en GitHub)
1. Un dataset **en CSV** (generado en el notebook).
2. Un notebook reproducible con:
   - Limpieza + EDA
   - Detección y tratamiento de outliers
   - Segmentación (if + apply)
   - Statistical Summary
3. Artefactos en el repo:
   - `README.md`
   - `data/clinic_transactions.csv`
   - `outputs/customer_metrics.csv`
   - `outputs/statistical_summary.md`
   - `figures/` (opcional: gráficos exportados)


## Diccionario de datos (CSV)

El archivo contiene **transacciones / citas** (1 fila = 1 cita facturada o reembolsada) con atributos del cliente.

| Columna | Tipo | Descripción |
|---|---|---|
| `transaction_id` | str | ID único de la transacción |
| `customer_id` | str | ID del cliente |
| `appointment_date` | datetime | Fecha de la cita/transacción |
| `service_type` | category | Tipo de servicio (consulta, laboratorio, etc.) |
| `appointment_channel` | category | Canal de agenda (Web/App/Call Center/Presencial) |
| `payment_method` | category | Método de pago |
| `amount_gross` | float | Valor bruto antes de descuentos |
| `discount_pct` | int | % de descuento aplicado |
| `amount_net` | float | Valor neto (negativo si es reembolso) |
| `is_refund` | int | 1 si es reembolso, 0 si no |
| `refund_reason` | str | Motivo de reembolso (si aplica) |
| `wait_time_min` | float | Tiempo de espera (minutos) |
| `service_duration_min` | float | Duración del servicio (minutos) |
| `satisfaction_score` | float | Satisfacción (esperado 1–10; contiene nulos/outliers) |
| `signup_date` | datetime | Fecha de registro del cliente |
| `age` | int | Edad (contiene valores sucios/outliers) |
| `gender` | category | Género (F/M/X) |
| `city` | category | Ciudad |
| `acquisition_channel` | category | Canal de adquisición |
| `plan` | category | Plan (Básico/Plus/Premium) |
| `has_insurance` | int | 1 si tiene seguro, 0 si no |
| `is_revenue` | int | 1 si `amount_net` > 0 (ingreso), 0 si no |


In [ ]:
# ============================================================
# 0) Setup del proyecto
# ============================================================

import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# (Opcional) Para ver más columnas en pantalla
pd.set_option("display.max_columns", 200)

# (Opcional) Semilla para reproducibilidad
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

print("Entorno listo.")


## Ejercicio 1 · Generar el dataset sintético (CSV)

En proyectos reales, a veces necesitas **simular datos** para probar análisis, dashboards o pipelines.

En este ejercicio:
- Generaremos un dataset realista de transacciones (citas) y lo guardaremos como CSV.
- Luego trabajaremos **solo** a partir del CSV (como en un proyecto real).


In [ ]:
# ============================================================
# Puedes encontrar el dataset en el archivo "datasets/sprint7_webinar22_clinic_transactions.csv"
# ============================================================


# df_preview = pd.read_csv(csv_file)
# df_preview.head()


## Ejercicio 2 · Cargar el CSV y revisar calidad de datos (EDA rápido)

En un proyecto real, lo primero es responder:

- ¿Cuántas filas/columnas tenemos?
- ¿Hay valores nulos? ¿En qué columnas?
- ¿Hay duplicados?
- ¿Los tipos de datos (fechas/números) son correctos?

**Meta:** terminar con un dataframe listo para análisis (con tipos correctos y flags útiles).


In [ ]:
# ============================================================
# 2) Cargar CSV y EDA rápido
# ============================================================

df = pd.read_csv(csv_file)

print("Shape:", df.shape)
display(df.head())

# Tipos de datos actuales
df.dtypes


In [ ]:
# Revisión de nulos y duplicados
missing = df.isna().mean().sort_values(ascending=False)
display(missing.head(10))

print("Duplicados (filas completas):", df.duplicated().sum())
print("Duplicados por transaction_id:", df["transaction_id"].duplicated().sum())


In [ ]:
# Convertir a datetime columnas de fecha
df["appointment_date"] = pd.to_datetime(df["appointment_date"], errors="coerce")
df["signup_date"] = pd.to_datetime(df["signup_date"], errors="coerce")

# Validar conversiones
print("Fechas inválidas (appointment_date):", df["appointment_date"].isna().sum())
print("Fechas inválidas (signup_date):", df["signup_date"].isna().sum())

df[["appointment_date","signup_date"]].describe(datetime_is_numeric=True)


## Ejercicio 3 · Construir métricas por cliente (base para análisis y segmentación)

En la práctica, la segmentación suele hacerse con variables agregadas.  
Vamos a construir un dataframe de métricas por cliente, por ejemplo:

- `num_visits`: número de citas
- `total_spend`: gasto total (solo ingresos, excluyendo reembolsos)
- `refund_count` y `refund_rate`
- `avg_ticket` y `max_ticket`
- `last_visit_date` y `recency_days` (días desde la última cita)
- KPIs operativos: `avg_wait_time`, `avg_duration`, `avg_satisfaction`

**Nota:** aquí aplicamos `groupby` + `agg`, que es una habilidad esencial.


In [ ]:
# ============================================================
# 3) Métricas por cliente (agregación)
# ============================================================

# Separar ingresos vs reembolsos
df["is_refund"] = df["is_refund"].astype(int)

df_revenue = df[df["amount_net"] > 0].copy()
df_refunds = df[df["amount_net"] < 0].copy()

print("Ingresos:", len(df_revenue), "| Reembolsos:", len(df_refunds))

# Fecha de corte del análisis (simula “hoy” en un proyecto)
as_of_date = df["appointment_date"].max()
as_of_date


In [ ]:
# Agregación por cliente
customer_metrics = (
    df.groupby("customer_id")
      .agg(
          num_visits=("transaction_id","nunique"),
          total_spend=("amount_net", lambda s: s[s>0].sum()),      # solo ingresos
          refund_count=("is_refund","sum"),
          avg_ticket=("amount_net", lambda s: s[s>0].mean()),
          max_ticket=("amount_net", lambda s: s[s>0].max()),
          last_visit_date=("appointment_date","max"),
          avg_wait_time=("wait_time_min","mean"),
          avg_duration=("service_duration_min","mean"),
          avg_satisfaction=("satisfaction_score","mean"),
          city=("city","first"),
          plan=("plan","first"),
          acquisition_channel=("acquisition_channel","first"),
          age=("age","first"),
          has_insurance=("has_insurance","first")
      )
      .reset_index()
)

# Recency en días
customer_metrics["recency_days"] = (as_of_date - customer_metrics["last_visit_date"]).dt.days

# Refund rate (por visitas)
customer_metrics["refund_rate"] = customer_metrics["refund_count"] / customer_metrics["num_visits"]

display(customer_metrics.head())
customer_metrics.describe()


### Mini-ejercicio (5 min)

1. Calcula el **percentil 90** de `total_spend` y úsalo para identificar los clientes “Top 10%”.
2. Crea una columna `is_top_spender` (1 si está en el Top 10%, 0 si no).

Pista: `customer_metrics["total_spend"].quantile(0.90)`


In [ ]:
# Solución (Mini-ejercicio)
p90 = customer_metrics["total_spend"].quantile(0.90)
customer_metrics["is_top_spender"] = (customer_metrics["total_spend"] >= p90).astype(int)

p90, customer_metrics["is_top_spender"].value_counts()


## 7.3.1 · Identificando valores atípicos con reglas estadísticas (aplicación práctica)

En el webinar teórico vimos dos reglas comunes:

- **IQR (Interquartile Range):** robusto para distribuciones sesgadas (típico en montos).
- **Z-score:** útil si la variable es aproximadamente normal (o tras transformar).

En un proyecto real, esto se traduce en:
1. Medir cuántos outliers hay.
2. Inspeccionar ejemplos (¿errores? ¿casos reales extremos?).
3. Decidir tratamiento (no siempre se eliminan).

En este ejercicio analizaremos outliers en:
- `amount_net` (en ingresos)
- `wait_time_min`
- `age` (calidad de datos)


In [ ]:
# ============================================================
# 7.3.1 A) Detección de outliers con IQR (ejemplo: total_spend)
# ============================================================

metric = "total_spend"

q1 = customer_metrics[metric].quantile(0.25)
q3 = customer_metrics[metric].quantile(0.75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

customer_metrics["outlier_iqr_total_spend"] = ((customer_metrics[metric] < lower) | (customer_metrics[metric] > upper)).astype(int)

print("IQR bounds:", lower, upper)
customer_metrics["outlier_iqr_total_spend"].value_counts()


In [ ]:
# Ver ejemplos de outliers (top 10 por total_spend)
outliers_spend = customer_metrics[customer_metrics["outlier_iqr_total_spend"] == 1].sort_values("total_spend", ascending=False)
display(outliers_spend.head(10))


In [ ]:
# ============================================================
# 7.3.1 B) Detección de outliers con Z-score (ejemplo: wait_time_min)
# ============================================================

# Para Z-score es mejor usar una variable por transacción (aquí: wait_time_min)
wait = df["wait_time_min"].dropna()

mean_w = wait.mean()
std_w = wait.std(ddof=0)

z = (df["wait_time_min"] - mean_w) / std_w
df["outlier_z_wait_time"] = (z.abs() > 3).astype(int)

df["outlier_z_wait_time"].value_counts()


In [ ]:
# Visualización rápida (histograma) para entender sesgo y outliers
plt.figure(figsize=(10,4))
plt.hist(df_revenue["amount_net"], bins=50)
plt.title("Distribución de amount_net (solo ingresos)")
plt.xlabel("amount_net")
plt.ylabel("frecuencia")
plt.show()

plt.figure(figsize=(10,4))
plt.hist(df["wait_time_min"], bins=50)
plt.title("Distribución de wait_time_min")
plt.xlabel("wait_time_min")
plt.ylabel("frecuencia")
plt.show()


## 7.3.2 · Cómo abordar valores atípicos según el contexto (aplicación práctica)

Regla de oro: **outlier ≠ error**.  
Un outlier puede ser:

- Un **error de captura** (edad = 999, satisfacción = 12)
- Un **evento real extremo** (una atención costosa)
- Un **sub-proceso diferente** (reembolsos: valores negativos esperados)

Aquí tomaremos decisiones típicas de negocio:

1. **Edad**: marcar valores fuera de rango y corregir a `NaN` para no distorsionar.
2. **Satisfacción**: forzar rango 1–10 y dejar `NaN` si es inválido.
3. **Montos**: usar *capping* (winsorization) en p99 para análisis agregado.
4. **Reembolsos**: NO mezclarlos como “outliers”; tratarlos como categoría separada.


In [ ]:
# ============================================================
# 7.3.2 A) Limpieza por reglas de negocio (edad y satisfacción)
# ============================================================

df_clean = df.copy()

# Edad válida: 18 a 90 (lo demás lo marcamos como NaN)
df_clean["age_clean"] = df_clean["age"].where(df_clean["age"].between(18, 90), np.nan)

# Satisfacción válida: 1 a 10 (lo demás NaN)
df_clean["satisfaction_clean"] = df_clean["satisfaction_score"].where(df_clean["satisfaction_score"].between(1, 10), np.nan)

print("Edad inválida:", df_clean["age_clean"].isna().sum(), "de", len(df_clean))
print("Satisfacción inválida:", df_clean["satisfaction_clean"].isna().sum(), "de", len(df_clean))


In [ ]:
# ============================================================
# 7.3.2 B) Capping de montos para análisis (p99 sobre ingresos)
# ============================================================

p99 = df_clean.loc[df_clean["amount_net"] > 0, "amount_net"].quantile(0.99)

df_clean["amount_net_capped"] = df_clean["amount_net"].copy()

# Solo capear ingresos (no tocar reembolsos)
mask_revenue = df_clean["amount_net_capped"] > 0
df_clean.loc[mask_revenue, "amount_net_capped"] = df_clean.loc[mask_revenue, "amount_net_capped"].clip(upper=p99)

print("p99 amount_net (ingresos):", p99)
df_clean.loc[mask_revenue, ["amount_net","amount_net_capped"]].describe()


In [ ]:
# Comparar distribución antes/después (ingresos)
plt.figure(figsize=(10,4))
plt.hist(df_clean.loc[mask_revenue, "amount_net"], bins=50)
plt.title("amount_net (antes) - solo ingresos")
plt.xlabel("amount_net")
plt.ylabel("frecuencia")
plt.show()

plt.figure(figsize=(10,4))
plt.hist(df_clean.loc[mask_revenue, "amount_net_capped"], bins=50)
plt.title("amount_net_capped (después) - solo ingresos")
plt.xlabel("amount_net_capped")
plt.ylabel("frecuencia")
plt.show()


## 7.3.3 · Segmentación de clientes con sentencias `if/elif/else` (proyecto)

Vamos a crear una segmentación simple tipo RFM:

- **Recency** (días desde la última visita)
- **Frequency** (num_visits)
- **Monetary** (total_spend)

Objetivo: producir una columna `segment_if` con categorías accionables:
- `VIP`
- `Leal`
- `Prometedor`
- `En riesgo`
- `Bajo valor`

Regla importante: la segmentación debe ser **explicable** y **reproducible**.


In [ ]:
# ============================================================
# 7.3.3 Segmentación con if/elif/else
# ============================================================

# Reconstruimos métricas usando df_clean (para usar montos cappeados si queremos)
df_rev_clean = df_clean[df_clean["amount_net_capped"] > 0].copy()

as_of_date = df_clean["appointment_date"].max()

customer_metrics2 = (
    df_clean.groupby("customer_id")
      .agg(
          num_visits=("transaction_id","nunique"),
          total_spend=("amount_net_capped", lambda s: s[s>0].sum()),
          last_visit_date=("appointment_date","max"),
          avg_satisfaction=("satisfaction_clean","mean")
      )
      .reset_index()
)

customer_metrics2["recency_days"] = (as_of_date - customer_metrics2["last_visit_date"]).dt.days

# Umbrales simples (pueden ajustarse)
p75_spend = customer_metrics2["total_spend"].quantile(0.75)
p75_visits = customer_metrics2["num_visits"].quantile(0.75)

def segment_customer_if(row: pd.Series) -> str:
    """Asigna un segmento basado en reglas RFM sencillas.

    Nota pedagógica: se usa if/elif/else para que sea claro y fácil de explicar.
    """
    recency = row["recency_days"]
    freq = row["num_visits"]
    monetary = row["total_spend"]
    sat = row["avg_satisfaction"]

    # VIP: reciente, frecuente y alto gasto
    if (recency <= 30) and (freq >= p75_visits) and (monetary >= p75_spend):
        return "VIP"

    # Leal: frecuente y relativamente reciente
    elif (recency <= 60) and (freq >= p75_visits):
        return "Leal"

    # Prometedor: reciente pero con menos frecuencia (o gasto moderado)
    elif (recency <= 30) and (freq < p75_visits):
        return "Prometedor"

    # En riesgo: no ha venido hace tiempo pero antes tuvo actividad
    elif (recency > 90) and (freq >= 3):
        return "En riesgo"

    # Bajo valor: poca frecuencia y poco gasto
    else:
        return "Bajo valor"

customer_metrics2["segment_if"] = customer_metrics2.apply(segment_customer_if, axis=1)

customer_metrics2["segment_if"].value_counts()


## 7.3.4 · Segmentación con `apply()` y funciones que integran IF para crear nuevas columnas

En proyectos reales, el “segmento” no es lo único que necesitamos.  
Frecuentemente creamos **múltiples variables** para activar acciones:

- `needs_follow_up` (¿requiere seguimiento?)
- `discount_eligible` (¿es elegible a incentivo?)
- `recommended_channel` (¿por dónde contactarlo?)

Vamos a construir una función que retorna un `pd.Series` con varias columnas nuevas y la aplicaremos con `apply()`.


In [ ]:
# ============================================================
# 7.3.4 apply() para crear varias columnas
# ============================================================

def enrichment_rules(row: pd.Series) -> pd.Series:
    """Reglas de enriquecimiento (múltiples flags) basadas en condiciones.

    Retorna un pd.Series con varias columnas:
    - needs_follow_up
    - discount_eligible
    - recommended_channel
    """
    recency = row["recency_days"]
    seg = row["segment_if"]
    sat = row["avg_satisfaction"]

    # 1) needs_follow_up: alto recency o satisfacción baja
    if (recency > 90) or (pd.notna(sat) and sat <= 6):
        needs_follow_up = 1
    else:
        needs_follow_up = 0

    # 2) discount_eligible: segmento “En riesgo” o “Bajo valor” con cierta actividad
    if seg in ["En riesgo", "Bajo valor"] and row["num_visits"] >= 2:
        discount_eligible = 1
    else:
        discount_eligible = 0

    # 3) recommended_channel: por defecto Email, pero si recency muy alto -> Call Center
    if recency > 120:
        recommended_channel = "Call Center"
    else:
        recommended_channel = "Email"

    return pd.Series({
        "needs_follow_up": needs_follow_up,
        "discount_eligible": discount_eligible,
        "recommended_channel": recommended_channel
    })

enriched = customer_metrics2.apply(enrichment_rules, axis=1)
customer_metrics2 = pd.concat([customer_metrics2, enriched], axis=1)

display(customer_metrics2.head())


### Mini-ejercicio (5 min)

1. Crea una columna `priority` con estos valores:
   - `Alta` si `needs_follow_up == 1` y `discount_eligible == 1`
   - `Media` si `needs_follow_up == 1` y `discount_eligible == 0`
   - `Baja` en cualquier otro caso

Hazlo de dos maneras:
- (A) con `if/elif/else` en una función
- (B) con `np.select`

Compara cuál te parece más legible para un equipo.


In [ ]:
# Solución (Mini-ejercicio)

def priority_if(row):
    if (row["needs_follow_up"] == 1) and (row["discount_eligible"] == 1):
        return "Alta"
    elif (row["needs_follow_up"] == 1) and (row["discount_eligible"] == 0):
        return "Media"
    else:
        return "Baja"

customer_metrics2["priority_if"] = customer_metrics2.apply(priority_if, axis=1)

conditions = [
    (customer_metrics2["needs_follow_up"] == 1) & (customer_metrics2["discount_eligible"] == 1),
    (customer_metrics2["needs_follow_up"] == 1) & (customer_metrics2["discount_eligible"] == 0),
]
choices = ["Alta", "Media"]

customer_metrics2["priority_npselect"] = np.select(conditions, choices, default="Baja")

customer_metrics2[["priority_if","priority_npselect"]].head()


## 7.3.5 · Redacción de un Statistical Summary (proyecto)

Un Statistical Summary es un texto corto (1–2 páginas) que:

1. Resume el dataset (tamaño, cobertura temporal).
2. Describe hallazgos clave (KPIs, distribuciones).
3. Documenta decisiones (tratamiento de outliers, supuestos).
4. Entrega conclusiones y próximos pasos.

Aquí crearemos una **plantilla automática** que toma métricas calculadas y genera un borrador en Markdown.


In [ ]:
# ============================================================
# 7.3.5 Generar un Statistical Summary en Markdown
# ============================================================

# KPIs globales
n_rows = len(df_clean)
n_customers = df_clean["customer_id"].nunique()

date_min = df_clean["appointment_date"].min().date()
date_max = df_clean["appointment_date"].max().date()

refund_rate_global = (df_clean["amount_net"] < 0).mean()
avg_ticket_global = df_clean.loc[df_clean["amount_net_capped"] > 0, "amount_net_capped"].mean()

# Distribución de segmentos
segment_dist = customer_metrics2["segment_if"].value_counts(normalize=True).round(4)

summary_md = f"""# Statistical Summary · Clínica Agendada

## 1. Dataset
- Filas (transacciones/citas): **{n_rows:,}**
- Clientes únicos: **{n_customers:,}**
- Ventana temporal: **{date_min} → {date_max}**

## 2. Calidad y preparación
- Se crearon columnas limpias:
  - `age_clean` (edad válida 18–90; inválidos a NaN)
  - `satisfaction_clean` (rango 1–10; inválidos a NaN)
- Para análisis de montos, se aplicó *capping* en p99 sobre ingresos (no reembolsos):
  - `amount_net_capped`

## 3. Hallazgos (KPIs)
- Tasa global de reembolsos (por transacción): **{refund_rate_global:.2%}**
- Ticket promedio (ingresos cappeados): **{avg_ticket_global:,.0f}**

## 4. Segmentación (RFM simplificado)
Distribución de segmentos (proporción de clientes):
{segment_dist.to_string()}

## 5. Recomendaciones
- Revisar outliers de espera (`wait_time_min`) para identificar cuellos de botella.
- Diseñar campañas por segmento:
  - VIP: beneficios premium / cross-sell
  - En riesgo: follow-up + incentivo controlado
  - Bajo valor: onboarding + educación del servicio

## 6. Próximos pasos
- Validar reglas con stakeholders (operaciones, finanzas).
- A/B test de incentivos por segmento.
"""

# Guardar el summary como artefacto de proyecto
summary_file = OUTPUTS_DIR / "statistical_summary.md"
summary_file.write_text(summary_md, encoding="utf-8")

print(f"Resumen guardado en: {summary_file}")
print(summary_md[:600] + "\n...")  # preview


## 7.4 · Almacenando y compartiendo análisis (publicación en GitHub)

En un proyecto real, tu análisis debe ser:
- **Reproducible** (otros pueden correrlo)
- **Versionado** (historial de cambios)
- **Compartible** (enlace a repo + README)

En este bloque vamos a:
1. Guardar artefactos (`outputs/`, `figures/`).
2. Crear `README.md` mínimo.
3. Inicializar repo Git, hacer commits y preparar el push a GitHub.
4. Ver el workflow típico en Google Colab.


In [ ]:
# ============================================================
# 7.4 A) Guardar outputs del proyecto
# ============================================================

# Guardar métricas por cliente
customer_metrics_file = OUTPUTS_DIR / "customer_metrics.csv"
customer_metrics2.to_csv(customer_metrics_file, index=False, encoding="utf-8")

print(f"Archivo guardado: {customer_metrics_file} | filas={len(customer_metrics2):,}")

# (Opcional) Guardar un gráfico como figura
plt.figure(figsize=(10,4))
plt.hist(customer_metrics2["total_spend"], bins=50)
plt.title("Distribución de total_spend por cliente (cappeado)")
plt.xlabel("total_spend")
plt.ylabel("frecuencia")

fig_path = FIG_DIR / "total_spend_hist.png"
plt.savefig(fig_path, dpi=150, bbox_inches="tight")
plt.show()

print(f"Figura guardada: {fig_path}")


In [ ]:
# ============================================================
# 7.4 B) Crear README.md y (opcional) requirements.txt
# ============================================================

readme_file = Path("README.md")
if not readme_file.exists():
    readme_file.write_text("""# Proyecto · Clínica Agendada (Sprint 7)

Este repositorio contiene un proyecto práctico de Data Analytics para:

- Analizar calidad de datos y EDA
- Detectar y tratar outliers (IQR/Z-score + contexto de negocio)
- Construir métricas por cliente (RFM simplificado)
- Segmentar clientes (if + apply)
- Redactar un Statistical Summary

## Estructura

- `data/clinic_transactions.csv`: dataset (sintético)
- `notebooks/`: notebook del proyecto
- `outputs/`: métricas y summary
- `figures/`: gráficos exportados

## Cómo ejecutar

1. Crear entorno (recomendado): `python -m venv .venv`
2. Instalar dependencias: `pip install -r requirements.txt`
3. Abrir y ejecutar el notebook.

## Resultados principales

Ver `outputs/statistical_summary.md`.
""", encoding="utf-8")
    print("README.md creado.")
else:
    print("README.md ya existe (no se sobreescribió).")

requirements_file = Path("requirements.txt")
if not requirements_file.exists():
    requirements_file.write_text("\n".join([
        "pandas",
        "numpy",
        "matplotlib"
    ]) + "\n", encoding="utf-8")
    print("requirements.txt creado.")
else:
    print("requirements.txt ya existe (no se sobreescribió).")


### 7.4.1–7.4.3 · GitHub para analistas + estructura de repos + workflow con Colab

A continuación tienes comandos típicos para publicar tu proyecto.

> Nota: estos comandos se ejecutan en terminal.  
> En Jupyter/Colab puedes ejecutarlos con `!` al inicio.

#### Paso 1: Crear estructura de repo (sugerida)

```
project/
  data/
  notebooks/
  outputs/
  figures/
  README.md
  requirements.txt
  .gitignore
```

#### Paso 2: Inicializar git y primer commit

```
git init
git add .
git commit -m "Initial commit: project skeleton + dataset + notebook"
```

#### Paso 3: Crear repo en GitHub y agregar remote

En GitHub: New repository → copia la URL (HTTPS).

```
git remote add origin https://github.com/<TU_USUARIO>/<TU_REPO>.git
git branch -M main
git push -u origin main
```

#### Paso 4: Workflow recomendado (GitHub Flow)

- Crea branch por cambio: `git checkout -b feature/segmentacion`
- Commits pequeños y descriptivos
- Pull Request → revisión → merge

#### En Google Colab (workflow típico)
1. Conecta Google Drive (opcional).
2. Clona tu repo: `!git clone https://github.com/<TU_USUARIO>/<TU_REPO>.git`
3. Ejecuta el notebook, genera outputs.
4. Commit y push desde Colab (requiere autenticación/token).

**Tip:** evita subir datos sensibles. En este proyecto el dataset es sintético, así que está OK subirlo.


## Takeaways

- La detección de outliers (IQR/Z-score) es el **inicio**, no el final: el contexto define la acción.
- Segmentación efectiva = reglas **claras**, variables agregadas y trazabilidad.
- Un buen Statistical Summary documenta **métricas + decisiones + supuestos**.
- Publicar en GitHub vuelve tu análisis **reproducible** y facilita colaboración.


## Cierre
**Kahoot de repaso (5 min)**
- Trabajamos en el proyecto "Clínica Agendada" de principio a fin.
- Generamos datos, limpiamos y detectamos segmentos.

**Reflexión:**
- ¿Qué fue lo más retador de integrar todas las habilidades (Python + Stats + Git)?
- ¿Cómo explicarías tus hallazgos a un director médico no técnico?

**Q&A y próximos pasos.**


## Siguientes Pasos
- **Próxima sesión:** Sprint 8 - Correlaciones y Relaciones entre variables.
- **Participación:** Completa el README de tu repositorio en GitHub.
- **Recordatorios:** Entrega el link de tu repo en la plataforma.
